# Imports and Data Loading

In [1]:
import sys
import numpy as np
sys.path.append('../')
from src.models.cosine_knn import CosineKNN
from src.models.jaccard import SimpleJaccard
from src.models.popular_recommender import PopularRecommender
import pandas as pd
from src.models.model import RedditGraph
from src.models.evaluator import Evaluator
import warnings
warnings.filterwarnings('ignore')

In [2]:
comments_to_subreddit = {}
data = []
users = []
with open('../data/out/users_comments.csv') as ucf, open('../data/out/comments.csv') as cf:
    for l in cf.readlines():
        c, sr, _, _ = l.split(',')
        comments_to_subreddit[c] = sr

    for l in ucf.readlines():
        u, c = l.split(',')
        data.append((u, comments_to_subreddit[c.strip()]))
df = pd.DataFrame(data, columns=['user', 'subreddit'])

# Our user:
In this notebook, we will make some subreddit recommendations for user 'lowpass'.

In [3]:
user = 'lowpass'

# Evaluation:
Creating an evaluation helper:

In [4]:
evaluator = Evaluator('../data/out/test_interactions.csv')

# Baseline recommender models:

## Popularity recommender:
Recommends popular subreddits user does not belong to

In [4]:
pop_rec = PopularRecommender(df)
pop_rec.recommend(user)

['politics']

In [5]:
pop_rec.recommend(user)

['politics']

## Jaccard similarity recommender:
Recommends subreddits by determining what similar users (determined by Jaccard similarity) belong to

In [6]:
jaccard_rec = SimpleJaccard(df)
jaccard_rec.recommend(user)

['learnprogramming']

In [7]:
jaccard_rec.recommend(user)

['learnprogramming']

## Cosine similarity KNN recommender:
Recommends subreddits by determining what similar users (determined by nearest-neighbor similarity in a KNN) belong to

In [8]:
knn_rec = CosineKNN(df)
knn_rec.user_recommend(user)

['beer']

In [9]:
knn_rec.user_recommend(user)

['beer']

# Interaction graph recommender models:

In [5]:
reddit_graph = RedditGraph('../config.json')

## Louvain community recommender:
Recommends subreddits by determining what similar users (determined by assigned Louvain community) belong to

In [4]:
reddit_graph.predict_louvain(user)

NameError: name 'reddit_graph' is not defined

## Centrality KNN community recommender:
Recommends subreddits by determining what similar users (determined by nearest neighbors of the following centrality metrics: pagerank, louvain, label propagation, degree) belong to

In [ ]:
reddit_graph.fit_knn('user', k=1)

In [ ]:
preds = reddit_graph.predict_knn(user, k=20)

In [8]:
evaluator.precision(user, preds)

{'Deusex', 'math', 'gaming', 'malefashionadvice', 'WTF', 'Cooking', 'wikipedia', 'corgi', 'nova', 'atheism', 'pics'}
set()


0.0

In [9]:
evaluator.recall(user, preds)

0.0